## Installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install transformers nlp datasets tensorflow-text tf-models-official pytorch

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import Dataset
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import keras, pickle
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import random
import json
import nltk
nltk.download('wordnet')

In [ ]:
! cp -av /content/drive/MyDrive/ChatBox ./dataset/

In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('dataset/intents.json').read()
intents = json.loads(data_file)

## Data pre-processing

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 9) #len(label_dict)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = tokenizer.tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

In [ ]:
documents[0]

## Dataset (prepare input for BERT)

In [ ]:
import pandas as pd
df = pd.DataFrame(documents)
print(df.head(10))
df[0] = df[0].apply(' '.join)
df.rename(columns={0: "text", 1: "label"},inplace=True)
df.head(10)

In [ ]:
possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

In [ ]:
df['label'] = df.label.replace(label_dict)

In [ ]:
! pip install datasets
from datasets import Dataset

In [ ]:
f_data = Dataset.from_pandas(df)
f_data[0]

In [ ]:
def preprocess(data):
    return tokenizer(data["text"], max_length = 15, padding="max_length", truncation=True)

In [ ]:
train_set = f_data.map(preprocess, batched=True, batch_size=len(f_data))

In [ ]:
train_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label']) 

In [ ]:
dataloader = torch.utils.data.DataLoader(train_set, batch_size=1)
next(iter(dataloader))

## Model Training

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 9) #len(label_dict)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1000,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    warmup_steps=10,
    weight_decay=0.0001,
    logging_dir='./logs',
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=train_set
)

In [ ]:
trainer.train()

In [ ]:
model = torch.load("results/checkpoint-10000/pytorch_model.bin")
torch.save(model, './drive/MyDrive/ChatBox/model1000')

## Predictions

In [ ]:
my_model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = len(label_dict))

checkpoint = torch.load("./drive/MyDrive/ChatBox/model1000C")
my_model.load_state_dict(checkpoint)
print(type(my_model))

In [ ]:
import json
import random
intents = json.loads(open('dataset/intents.json').read())
words = pickle.load(open('dataset/words.pkl','rb'))
classes = pickle.load(open('dataset/classes.pkl','rb'))

In [ ]:
for i in train_set:
  #print(i["input_ids"],i["attention_mask"])
  #print(torch.stack((i["input_ids"],i["attention_mask"]),0))
  with torch.no_grad():
      logits = my_model(torch.stack((i["input_ids"],i["attention_mask"]),0)).logits
  print("XX----",logits[0])

  print("label=",i["label"].item())


  predicted_class_id = logits[0].argmax().item()
  print(predicted_class_id)

## Bot Response

In [ ]:
def inits(text):
  out = tokenizer(text, max_length = 15, padding="max_length", truncation=True)
  return torch.tensor(out.input_ids), torch.tensor(out.attention_mask)

def getPreds(ids,att,model):
  with torch.no_grad():
    logits = model(torch.stack((ids,att),0)).logits
  return logits[0].argmax().item()

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = inits(text)
    preds = getPreds(ints,my_model)
    res = getResponse(ints, intents)
    return res

In [ ]:
text = "what can you do"
int1, int2 = inits(text)
getPreds(int1,int2,my_model)